## Monitoring Scoring Piplein 

In what follows, we will instrument monitroing agents to montiroing the 19 models

## Generate some test data

This is an example of scoring data and monitoring with datarobot

In [1]:
import json
import pandas as pd 
import yaml
import datarobot as dr
from datarobot_mlops.mlops import MLOps
import time
import yaml 
import numpy as np 

import numpy as np 
def generate_data(n = 100):
    arr = ["low", "medium", "high"]
    predictions = np.random.uniform(1000,10000, [n, 19])
    exposureType = np.random.choice(["exposure type a", "exposure type b", "exposure type c"], size=n, p = [1/3 ,1/3, 1/3], replace=True)
    claimNumber = np.random.randint(11111, 99999, size = n)
    payload = pd.DataFrame( dict( claimNumber = claimNumber, exposureType = exposureType))
    predictions = pd.DataFrame(predictions, columns = [f"quantile {q}" for q in list(range(5,100,5))])
    payload["claimantInfo"] = json.dumps( dict( field1 = "field1", field2 = "field2"))
    return payload.join(predictions)

In [2]:
generate_data(5)

,claimNumber,exposureType,claimantInfo,quantile 5,quantile 10,quantile 15,quantile 20,quantile 25,quantile 30,quantile 35,...,quantile 50,quantile 55,quantile 60,quantile 65,quantile 70,quantile 75,quantile 80,quantile 85,quantile 90,quantile 95
0,93427,exposure type a,"{""field1"": ""field1"", ""field2"": ""field2""}",2356.676363,1734.914326,4435.691088,1954.557454,5516.374353,2687.534884,9346.690425,...,6584.845968,7403.641914,3993.399006,8937.289116,3263.954197,7311.598293,2000.298538,2116.120916,1106.008403,3833.482030
1,79990,exposure type b,"{""field1"": ""field1"", ""field2"": ""field2""}",3901.897581,3083.171804,6225.051290,9168.096430,9908.916362,9580.781276,7377.153415,...,8790.992957,2749.789049,4868.241981,5966.265905,1735.991214,8361.625546,2870.211045,6751.127667,8137.719174,2507.782011
2,54200,exposure type c,"{""field1"": ""field1"", ""field2"": ""field2""}",5035.269443,3677.884292,9274.073247,3504.351424,4917.300399,7096.029967,9050.830322,...,4021.040485,9816.838418,2086.959731,8613.769948,4450.509117,6151.054753,5604.918318,1381.631797,8285.270786,3863.660516
3,97848,exposure type b,"{""field1"": ""field1"", ""field2"": ""field2""}",6817.931340,2743.727152,8184.523792,4823.454453,8572.139628,2777.474320,6069.732990,...,3607.777870,2835.300046,2751.393485,8189.979102,7879.732856,9625.201012,4419.880939,3457.147063,5112.996433,3866.219294
4,95937,exposure type a,"{""field1"": ""field1"", ""field2"": ""field2""}",9486.767911,4928.045653,3745.281345,4625.169551,3209.790186,2884.808319,3676.778641,...,4979.922412,5108.577721,2862.453425,8581.792940,4305.856148,7993.450579,8911.929135,1452.626051,6712.099484,6700.924544


## Using DataRobot Monitoring Agents + Filesystem as Spooler

The way remote monitoring works.  

* The python SDK is used to spool data to a spooler.  

* A monitoring agent is running somehwere - checking the spooler for new data and relaying that data to DataRobot.  

In this example, we use the filestystem as spooler.  

Please see details for other configiruations in the [DataRobot documentation](https://docs.datarobot.com/en/docs/mlops/deployment/mlops-agent/monitoring-agent/spooler.html)



In [46]:
dep = dr.Deployment.get(deployment_conf[0]["deployment_id"])
service_stats = dep.get_service_stats()
prediction_count = service_stats.metrics.get("totalPredictions")
print(prediction_count)

10000


In [ ]:
from pathlib import Path
from datarobot_mlops.mlops import MLOps
import os 
import glob 
import subprocess   
import time 
import json

## thia is the spooler director that we are creating on the fly
spooler_dir = Path("/tmp/ta")
spooler_dir.mkdir(exist_ok = True)
## the environment variables are a must for the client created on line 20 of this cell
## and for the agent that gets started in a few cells
os.environ["MLOPS_SERVICE_URL"] = "https://app.datarobot.com"
os.environ['MLOPS_API_TOKEN'] = os.environ["DATAROBOT_API_TOKEN"]
os.environ['MLOPS_AGENT_VERIFY_SSL'] = "true"
## the monitoring agent is a java service, and in order to run, we need JAVA_HOME set so it knows where to find java. 
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk/"
## since we are using the filesysten, we need to tell the client where to spool and where the agent needs to look.
os.environ["MLOPS_SPOOLER_TYPE"]="FILESYSTEM"
## we also need to specify the location of the spooler.
os.environ["MLOPS_FILESYSTEM_DIRECTORY"] = "/tmp/ta"
 
mlops = MLOps().init()
start = time.time() 
payload = generate_data(10000)
features_df = payload.drop
end = time.time() 
for deployment in deployment_conf:
    prediction_data = payload[deployment["tag"]].tolist()
    deployment_id = deployment["deployment_id"]
    model_id = deployment["model_id"]
    mlops.report_deployment_stats(len(prediction_data), (end - start)*1000, deployment_id = deployment_id, model_id = model_id) 
    mlops.report_predictions_data(predictions = prediction_data, deployment_id = deployment_id, model_id = model_id)
mlops.shutdown()

In [38]:
## start the agent!  this will use the environments variables set up above
agents_dir = glob.glob("./datarobot_mlops*").pop(0)
try: 
    os.remove(os.path.join(agents_dir, "bin", "PID.agent"))
except Exception as e:
    print(e)

subprocess.call("{}/bin/start-agent.sh".format(agents_dir))

INFO: MLOPS_AGENT_CONFIG_YAML=/home/notebooks/storage/datarobot_mlops_package-11.0.1/conf/mlops.agent.conf.yaml
INFO: MLOPS_AGENT_LOG_PROPERTIES=/home/notebooks/storage/datarobot_mlops_package-11.0.1/conf/mlops.log4j2.properties
INFO: MLOPS_AGENT_JVM_OPT=-Xmx1G
INFO: AGENT_CLASSPATH='/home/notebooks/storage/datarobot_mlops_package-11.0.1/lib/spooler-kafka-11.0.1.jar:/home/notebooks/storage/datarobot_mlops_package-11.0.1/lib/spooler-pubsub-11.0.1.jar:/home/notebooks/storage/datarobot_mlops_package-11.0.1/lib/spooler-rabbitmq-11.0.1.jar:/home/notebooks/storage/datarobot_mlops_package-11.0.1/lib/spooler-sqs-11.0.1.jar:/home/notebooks/storage/datarobot_mlops_package-11.0.1/lib/mlops-agent-11.0.1.jar'
INFO: AGENT_LOG_PATH=/home/notebooks/storage/datarobot_mlops_package-11.0.1/logs/mlops.agent.log

Running MLOps-Agent as a service


DataRobot MLOps-Agent is running.


0

## Give it some time to report back predictions

in a scheduled run, we need to block to make sure all predictions are reported back, otherwise the job will terminate before the spooler flushes all records to datarobot

In [39]:
predictions_reported = service_stats.metrics.get("totalPredictions") - prediction_count 
# predictions_reported != payload.shape[0]
while predictions_reported != payload.shape[0]:
    service_stats = dep.get_service_stats()
    predictions_reported = service_stats.metrics.get("totalPredictions") - prediction_count 
print("all predictions reported")
print(prediction_count)
print(service_stats.metrics.get("totalPredictions"))


all predictions reported
0
10000
